In [12]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd 
import keras
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import tensorflow as tf

from sklearn.model_selection import KFold


#print how many rows and columns are there in the csv file
df = pd.read_csv('train-balanced-sarcasm.csv')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')


There are 1010826 rows and 10 columns


In [25]:
#show the columns to see which ones we would need to use, and make a subdataframe with columns we need
print(df.columns)
df_useful_x = df[['comment']]
df_useful_y = df[['label']]
# convert each dataframe into 2D numpy arrays
x= df_useful_x.to_numpy()
y= df_useful_y.to_numpy()

Index(['label', 'comment', 'author', 'subreddit', 'score', 'ups', 'downs',
       'date', 'created_utc', 'parent_comment'],
      dtype='object')


In [24]:
x


array([['NC and NH.'],
       ['You do know west teams play against west teams more than east teams right?'],
       ["They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1"],
       ...,
       ['Perhaps this is an atheist conspiracy to make Christians look bad?'],
       ['The Slavs got their own country - it is called Kosovo'],
       ['values, as in capitalism .. there is good money in imprisoning people ..']],
      dtype=object)

In [26]:
y

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])